In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# load libraries
from FRVRS import nu, fu
from datetime import date, timedelta
from numpy import nan, isnan
from os import listdir as listdir, makedirs as makedirs, path as osp, remove as remove, sep as sep, walk as walk
from pandas import (
    CategoricalDtype, DataFrame, Index, NaT, Series, concat, get_dummies, isna, notnull, read_csv, read_excel,
    to_datetime, to_numeric
)
from re import split, search, sub, MULTILINE
from scipy.stats import f_oneway, ttest_ind, kruskal, norm
import itertools
from matplotlib import colors as colors, pyplot as plt
import numpy as np
import os
import re
import statsmodels.api as sm

today = date.today()

In [3]:

# load data frames
data_frames_dict = nu.load_data_frames(
    **{'verbose': True, '20240521truncated_csv_stats_df': ''}
)
csv_stats_df = data_frames_dict['20240521truncated_csv_stats_df'].copy()
print(csv_stats_df.shape)

No pickle exists for 20240521truncated_csv_stats_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/20240521truncated_csv_stats_df.csv.
(158663, 117)


In [7]:

# Add the sim environment column
new_column_name = 'encounter_layout'
encounter_layouts_list = ['Desert', 'Jungle', 'Submarine', 'Urban']
for (session_uuid, scene_id), scene_df in csv_stats_df.groupby(fu.scene_groupby_columns):
    mask_series = ~scene_df.patient_id.isnull()
    spl = sorted(scene_df[mask_series].patient_id.unique())
    for env_str in encounter_layouts_list:

        # Assume no Unity suffix
        patients_list = eval(f'fu.{env_str.lower()}_patients_list')
        if all(map(lambda p: p in spl, patients_list)):
            
            csv_stats_df.loc[scene_df.index, new_column_name] = env_str
display(csv_stats_df.groupby([new_column_name], dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
encounter_layout,
Desert,40678
Jungle,25582
Submarine,50198
Urban,42150
NaN,55


In [8]:

for df_name in dir():
    if df_name.endswith('_df'):
        if 'encounter_layout' in eval(f'{df_name}.columns'):
            print(df_name)

csv_stats_df


In [26]:

mask_series = ~session_df.encounter_layout.isnull()
session_df[mask_series].encounter_layout

35520    Desert
35521    Desert
35522    Desert
35523    Desert
35524    Desert
          ...  
35592    Desert
35593    Desert
35594    Desert
35595    Desert
35597    Desert
Name: encounter_layout, Length: 72, dtype: object

In [20]:

print(base_point)
print(neighbors_list)
print(nearest_neighbor)
print(euclidean_distance)

(8.131, 0, -28.682)
[(0.3, 0.0, -7.3)]
(0.3, 0.0, -7.3)
22.770913134962328


In [17]:

columns_list = ['action_type', 'action_tick', 'location_id', 'participant_id', 'encounter_layout', 'session_uuid']
session_df[mask_series][columns_list]

,action_type,action_tick,location_id,participant_id,encounter_layout,session_uuid
35563,TELEPORT,1369817,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35564,TELEPORT,1370395,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35565,TELEPORT,1372069,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35566,TELEPORT,1377061,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35567,TELEPORT,1378242,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35568,TELEPORT,1382824,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35582,TELEPORT,1443694,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35583,TELEPORT,1451294,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35585,TELEPORT,1461607,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c
35597,TELEPORT,1497960,"(0.3, 0.0, -7.3)",2024201,Desert,23081f6e-875e-44f5-8bd0-edc3905f5c2c


In [ ]:

# Get all the Open World JSON Stats into one data frame
logs_path = osp.join(nu.data_folder, 'logs', 'Human_Sim_Metrics_Data_4-12-2024')
directories_list = listdir(logs_path)

In [ ]:

json_stats_df = DataFrame([])
for dir_name in directories_list:
    
    # Add the JSONs to the data frame
    folder_path = osp.join(logs_path, dir_name)
    
    # Iterate over the files in the current subdirectory
    for file_name in listdir(folder_path):
        
        # If the file is a JSON file
        if file_name.endswith('.json'):
            
            # Create a data frame from the flattened dictionary
            json_path = osp.join(folder_path, file_name)
            with open(json_path, 'r') as f: file_json = json.load(f)
            row_dict = {
                'json_file_subpath': folder_path,
                'json_file_name': file_name
            }
            flattened_json_dict = nu.get_row_dictionary(file_json, row_dict=row_dict, key_prefix='')
            
            # You've got to clean the Session IDs
            session_uuid, participant_id = dir_name.split('_')
            flattened_json_dict['session_uuid'] = session_uuid
            flattened_json_dict['participant_id'] = int(participant_id)
            df = DataFrame(flattened_json_dict, index=Index([0]))
            
            # Append the data frame for the current subdirectory to the main data frame and break the participant ID loop
            json_stats_df = concat([json_stats_df, df], axis='index')
            
json_stats_df = json_stats_df.reset_index(drop=True)


### Drop boring columns

In [ ]:

# Check for proper ingestion (duplicate file ingestion, et al)
assert len(json_stats_df.columns) > 4, "Nothing ingested"
assert json_stats_df.participant_id.nunique() == 26, f"Participant count should be 26, it's {json_stats_df.participant_id.nunique()} instead"

In [ ]:

# Save the json stats dataset
nu.save_data_frames(**{f'{today.year}{today.month:02d}{today.day:02d}json_stats_df': json_stats_df}, verbose=True)


### Get the nearest neighbor location

In [ ]:

def get_action_tick_by_position(csv_stats_df, json_stats_df, participant_id, session_uuid, include_y=False):
    on_columns = ['participant_id', 'session_uuid']
    
    # Get the location (CSV) mask
    location_columns = on_columns + ['player_location_location']
    assert set(location_columns).issubset(
        set(csv_stats_df.columns)
    ), f"csv_stats_df is missing these location columns: {set(location_columns).difference(set(csv_stats_df.columns))}"
    location_mask_series = ~csv_stats_df.player_location_location.isnull()
    assert location_mask_series.any(), f"You don't have any player location data in csv_stats_df"
    location_session_mask = (csv_stats_df.participant_id == participant_id) & (csv_stats_df.session_uuid == session_uuid)
    assert location_session_mask.any(), f"You don't have any CSV data for participant #{participant_id} session #{session_uuid}"
    location_mask_series &= location_session_mask
    assert location_mask_series.any(), f"You don't have any player location data for participant #{participant_id} session #{session_uuid}"
    
    # Get the position (JSON) mask
    position_prefix_str = 'configData_playerData_startPosition_'
    if include_y:
        xyz_list = ['x', 'y', 'z']
    else:
        xyz_list = ['x', 'z']
    position_columns = on_columns + [f'{position_prefix_str}{d}' for d in xyz_list]
    assert set(position_columns).issubset(
        set(json_stats_df.columns)
    ), f"json_stats_df is missing these columns: {set(position_columns).difference(set(json_stats_df.columns))}"
    position_session_mask = (json_stats_df.participant_id == participant_id) & (json_stats_df.session_uuid == session_uuid)
    assert position_session_mask.any(), f"You don't have any JSON data for participant #{participant_id} session #{session_uuid}"
    
    # Get the base point (JSON)
    mask_series = False
    mask_series |= json_stats_df.configData_narrative_narrativeSections11_teleportPointsToActivate.isin(['Freeform'])
    mask_series |= json_stats_df.configData_narrative_narrativeSections12_teleportPointsToActivate.isin(['Freeform'])
    mask_series |= json_stats_df.configData_narrative_narrativeSections13_teleportPointsToActivate.isin(['Freeform'])
    df = json_stats_df[position_session_mask]
    position_x = df[mask_series][f'{position_prefix_str}x'].squeeze()
    if include_y:
        position_y = df[mask_series][f'{position_prefix_str}y'].squeeze()
    position_z = df[mask_series][f'{position_prefix_str}z'].squeeze()
    if include_y:
        base_point = (position_x, position_y, position_z)
    else:
        base_point = (position_x, position_z)
    
    # Create the left side of the merge
    csv_locations_df = csv_stats_df[location_mask_series][location_columns]
    for i, d in enumerate(xyz_list):
        csv_locations_df[f'location_{d}'] = csv_locations_df.player_location_location.map(lambda coord_str: eval(coord_str)[i])
    csv_locations_df = csv_locations_df.drop(columns=['player_location_location']).drop_duplicates()
    
    # Get the right side of the merge
    mask_series = ~json_stats_df[f'{position_prefix_str}x'].isnull()
    if include_y:
        mask_series &= ~json_stats_df[f'{position_prefix_str}y'].isnull()
    mask_series &= ~json_stats_df[f'{position_prefix_str}z'].isnull()
    assert set(position_columns).issubset(
        set(json_stats_df.columns)
    ), f"You're missing {set(position_columns).intersection(set(json_stats_df.columns))} as position columns"
    prefix_str = 'configData_playerData_startP'
    json_positions_df = json_stats_df[mask_series][position_columns].rename(
        columns={cn: cn.replace(prefix_str, 'p') for cn in position_columns if cn.startswith(prefix_str)}
    ).drop_duplicates()

    # Merge the two coordinate datasets
    on_columns = sorted(set(csv_locations_df.columns).intersection(set(json_positions_df.columns)))
    print(csv_locations_df.shape)
    print(json_positions_df.shape)
    merge_df = csv_locations_df.merge(json_positions_df, on=on_columns, how='inner').drop_duplicates()
    print(merge_df.shape)
    
    # Get the neighbors list
    neighbors_list = [(x, y, z) for (x, y, z), _ in csv_locations_df.groupby(['location_x', 'location_y', 'location_z'])]
    nearest_neighbor = nu.get_nearest_neighbor(base_point, neighbors_list)
    euclidean_distance = nu.get_euclidean_distance(base_point, nearest_neighbor)

In [ ]:

eval_json_df = DataFrame([])
folder_path = osp.join(fu.data_folder, 'json')

# Iterate over the files in the current subdirectory
for file_name in listdir(folder_path):
    
    # If the file is a JSON file
    if file_name.endswith('.json'):
        
        # Create a data frame from the flattened dictionary
        json_path = osp.join(folder_path, file_name)
        with open(json_path, 'r') as f: file_json = json.load(f)
        row_dict = {
            'json_file_name': file_name
        }
        flattened_json_dict = nu.get_row_dictionary(file_json, row_dict=row_dict, key_prefix='')
        df = DataFrame(flattened_json_dict, index=Index([0]))
        
        # Append the data frame for the current subdirectory to the main data frame and break the participant ID loop
        eval_json_df = concat([eval_json_df, df], axis='index')
        
eval_json_df = eval_json_df.reset_index(drop=True)

In [ ]:

eval_json_df.T

In [ ]:

# Get the teleport columns
mask_series = eval_json_df.columns.map(lambda cn: '_playerTeleportation_teleportPosition_' in cn)
teleport_columns = eval_json_df.columns[mask_series]
if len(teleport_columns) > 2:
    for column_name in sorted(teleport_columns):
        print(column_name)
elif len(teleport_columns):
    display(eval_json_df[teleport_columns].drop_duplicates())

In [ ]:

# Get the teleport columns
mask_series = eval_json_df.columns.map(lambda cn: 'teleportPointsToActivate' in cn)
teleport_columns = eval_json_df.columns[mask_series]
if len(teleport_columns) > 2:
    for column_name in sorted(teleport_columns):
        print(column_name)
elif len(teleport_columns):
    display(eval_json_df[teleport_columns].drop_duplicates())

In [ ]:

# Get all the teleport point names
for i in range(14):
    column_name = f'narrative_narrativeSections{i:02d}_teleportPointsToActivate'
    if column_name in eval_json_df:
        mask_series = ~eval_json_df[column_name].isnull()
        print( column_name, sorted(eval_json_df[mask_series][column_name].unique()))

In [ ]:

for (participant_id, session_uuid), session_df in csv_stats_df.groupby(['participant_id', 'session_uuid']):
    mask_series = (session_df.action_type == 'TELEPORT') & ~session_df.location_id.isnull()
    neighbors_list = [eval(location_id) for location_id in session_df[mask_series].location_id]
    nearest_neighbor = nu.get_nearest_neighbor(base_point, neighbors_list)
    euclidean_distance = nu.get_euclidean_distance(base_point, nearest_neighbor)
    break

In [ ]:

euclidean_distance

In [ ]:

mask_series = session_df.location_id.isin([str(nearest_neighbor)])
df = session_df[mask_series]
print(sorted(df.action_tick.unique()))
print(sorted(df.action_type.unique()))
df.T

In [ ]:

# Submarine: (0.02, 0, -13.5)
# Urban: (13.126, 0, 21.61)
# Jungle: (0.7, 0, 5.45)
# Desert: (8.131, 0, -28.682)
activate_columns = eval_json_df.columns[eval_json_df.columns.map(
    lambda cn: bool(re.search(r'narrative_narrativeSections\d+_teleportPointsToActivate', cn))
)]
mask_series = False
for cn in activate_columns:
    mask_series |= eval_json_df[cn].isin(['Freeform'])
teleport_columns = eval_json_df.columns[eval_json_df.columns.map(
    lambda cn: bool(re.search(r'narrative_narrativeSections\d+_playerTeleportation_teleportPosition_(x|y|z)', cn))
)]
columns_list = sorted(eval_json_df[mask_series][teleport_columns].dropna(axis='columns', how='all').columns)
description_columns = [cn for cn in sorted(eval_json_df.columns[eval_json_df.columns.map(
    lambda cn: bool(re.search(r'narrative_narrativeSections\d+_sectionDescription', cn))
)]) if cn.replace('_sectionDescription', '_playerTeleportation_teleportPosition_x') in columns_list]
activate_columns = [cn for cn in sorted(eval_json_df.columns[eval_json_df.columns.map(
    lambda cn: bool(re.search(r'narrative_narrativeSections\d+_teleportPointsToActivate', cn))
)]) if cn.replace('_teleportPointsToActivate', '_playerTeleportation_teleportPosition_x') in columns_list]
columns_list = sorted(columns_list + description_columns + activate_columns) + ['json_file_name']
eval_json_df[mask_series][columns_list].dropna(axis='index', how='all').T

In [ ]:

eval_json_df[mask_series].narrative_narrativeSections14_sectionDescription.tolist()

In [ ]:

# Get all the teleport point names
for i in range(14):
    column_name = f'configData_narrative_narrativeSections{i:02d}_teleportPointsToActivate'
    if column_name in json_stats_df:
        mask_series = ~json_stats_df[column_name].isnull()
        print( column_name, sorted(json_stats_df[mask_series][column_name].unique()))

In [ ]:

# Get list of all start position x, y, and z coordinates
prefix_str = 'configData_playerData_startPosition_'
print(f"\nThese are the values I can see for {' '.join(prefix_str.split('_')).strip()}:")
xyz_list = ['x', 'y', 'z']
for cn in [f'{prefix_str}{dim}' for dim in xyz_list]:
    position_mask_series = ~json_stats_df[cn].isnull()
    print(cn.split('_')[-1], set(json_stats_df[position_mask_series][cn]))

In [ ]:

columns_list = ['configData_teleportPointOverride']
display(json_stats_df[columns_list].drop_duplicates())

In [ ]:

# Get the narrativeConfig columns
mask_series = json_stats_df.columns.map(lambda cn: 'narrative' in cn)
config_columns = json_stats_df.columns[mask_series]
if len(config_columns) > 2:
    for column_name in sorted(config_columns):
        print(column_name)
elif len(config_columns):
    display(json_stats_df[config_columns].drop_duplicates())

In [ ]:

# Get columns that contain an x or z coordinate
xyz_list = ['x', 'y', 'z']
mask_series = json_stats_df.columns.map(lambda cn: any(map(lambda dim: cn.endswith(f'_{dim}'), xyz_list)))
xyz_columns = json_stats_df.columns[mask_series]
# print(xyz_columns)
location_regex = re.compile(r'-?\d+\.\d*')
location_columns = sorted(nu.get_regexed_columns(json_stats_df[xyz_columns], search_regex=location_regex))
if len(location_columns) > 2:
    for column_name in sorted(location_columns):
        print(column_name)
elif len(location_columns):
    display(json_stats_df[location_columns].drop_duplicates())

In [ ]:


location_mask_series = ~csv_stats_df.player_location_location.isnull()
location_columns = on_columns + ['player_location_location']
assert set(location_columns).issubset(set(csv_stats_df.columns)), "You've got the wrong location columns"
left_df = csv_stats_df[location_mask_series][location_columns]
for i, d in enumerate(xyz_list):
    left_df[f'location_{d}'] = left_df.player_location_location.map(lambda coord_str: eval(coord_str)[i])
left_df = left_df.drop(columns=['player_location_location'])

In [ ]:

right_gb = right_df.groupby(on_columns)
left_gb = left_df.groupby(on_columns)
tuples_list = sorted(set(right_gb.groups.keys()).intersection(set(left_gb.groups.keys())))
location_groupby_columns = [f'location_{d}' for d in xyz_list]
rows_list = []
for key_tuple in tuples_list:
    position_mask_series = (right_df.participant_id == key_tuple[0]) & (right_df.session_uuid == key_tuple[1])
    position_x = right_df[position_mask_series].position_x.squeeze()
    position_y = right_df[position_mask_series].position_y.squeeze()
    position_z = right_df[position_mask_series].position_z.squeeze()
    # display(right_df[position_mask_series].drop_duplicates())
    base_point = (position_x, position_y, position_z)
    
    location_mask_series = (left_df.participant_id == key_tuple[0]) & (left_df.session_uuid == key_tuple[1])
    neighbors_list = [(x, y, z) for (x, y, z), _ in left_df[location_mask_series].groupby(location_groupby_columns)]
    nearest_neighbor = nu.get_nearest_neighbor(base_point, neighbors_list)
    euclidean_distance = nu.get_euclidean_distance(base_point, nearest_neighbor)
    row_dict = {
        'base_point': base_point,
        'nearest_neighbor': nearest_neighbor,
        'euclidean_distance': euclidean_distance,
    }
    base_point = (position_x, position_z)
    neighbors_list = [(x, z) for (x, y, z), _ in left_df[location_mask_series].groupby(location_groupby_columns)]
    nearest_neighbor = nu.get_nearest_neighbor(base_point, neighbors_list)
    euclidean_distance = nu.get_euclidean_distance(base_point, nearest_neighbor)
    row_dict['euclidean_distance_xz'] = euclidean_distance
    rows_list.append(row_dict)
euclidean_distance_df = DataFrame(rows_list)

In [ ]:

# [cn for cn in df.columns if 'position' in cn.lower()]
mask_series = (df.configData_narrative_narrativeSections00_sectionDescription == 'Initialization')
# df.configData_scene.unique() == ['sim-urban-sanitized', 'sim-desert', 'sim-jungle']
df = df[mask_series].drop_duplicates()

In [ ]:

# Calculate the standard deviation for each column
std_dev = df.std()

# Get the indices of columns with zero variance
zero_variance_cols = std_dev[std_dev == 0].index

# Remove columns with zero variance
column_count = df.shape[1]
df.drop(columns=zero_variance_cols, inplace=True)
print(f"Dropped {int(column_count - df.shape[1])} JSON columns with zero variance")

# Assess the number of unique values (excluding NaNs) in each column
unique_values = df.nunique(dropna=True)

# Isolate the indices of columns with fewer than two unique values
few_unique_cols = unique_values[unique_values < 2].index

# Remove columns with fewer than two unique values
column_count = df.shape[1]
df.drop(columns=few_unique_cols, inplace=True)
print(f"Dropped {int(column_count - df.shape[1])} JSON columns with fewer than two unique values")

In [ ]:

df

In [ ]:

free_regex = re.compile('free', re.IGNORECASE)
free_columns = sorted(nu.get_regexed_columns(json_stats_df, search_regex=free_regex))
if len(free_columns) > 2:
    for column_name in sorted(free_columns):
        print(column_name)
else:
    display(json_stats_df[free_columns].drop_duplicates())

In [ ]:

adept_regex = re.compile('adept', re.IGNORECASE)
adept_columns = sorted(nu.get_regexed_columns(json_stats_df, search_regex=adept_regex))
if len(adept_columns) > 2:
    for column_name in sorted(adept_columns):
        print(column_name)
else:
    display(json_stats_df[adept_columns].drop_duplicates())

In [ ]:

soartech_regex = re.compile('soartech', re.IGNORECASE)
soartech_columns = sorted(nu.get_regexed_columns(json_stats_df, search_regex=soartech_regex))
if len(soartech_columns) > 2:
    for column_name in sorted(soartech_columns):
        print(column_name)
else:
    display(json_stats_df[soartech_columns].drop_duplicates())

In [ ]:

for column_name in sorted(set(adept_columns).intersection(set(soartech_columns))):
    print(column_name)

In [ ]:

for i in range(14):
    column_name = f'configData_narrative_narrativeSections{i:02d}_sectionDescription'
    if column_name in json_stats_df:
        mask_series = ~json_stats_df[column_name].isnull()
        print(column_name, sorted(json_stats_df[mask_series][column_name].unique()))

In [ ]:

for column_name in sorted([cn for cn in json_stats_df.columns if any(map(lambda x: x in cn.lower(), ['start', 'intro', 'free', 'form']))]):
    print(column_name)

In [ ]:

for column_name in sorted([cn for cn in json_stats_df.columns if any(map(lambda x: x in cn.lower(), ['position']))]):
    print(column_name)

In [ ]:

sorted(fu.location_id_columns_list)

In [ ]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_csv_stats_df=''
)
csv_stats_df = data_frames_dict['metrics_evaluation_open_world_csv_stats_df'].copy()

In [ ]:

location_regex = re.compile(r'\((?:-?\d{1,2}\.\d, ){2}-?\d{1,2}\.\d\)')
location_columns = sorted(nu.get_regexed_columns(csv_stats_df, search_regex=location_regex))
if len(location_columns) > 2:
    for column_name in sorted(location_columns):
        print(column_name)
elif len(location_columns):
    display(json_stats_df[location_columns].drop_duplicates())

In [ ]:

# Columns to merge the JSON stats dataset with the CSV stats on
on_columns = sorted(set(csv_stats_df.columns).intersection(set(json_stats_df.columns)))
print(on_columns)

In [ ]:

sorted(right_df.position_y.unique())

In [ ]:

sorted(left_df.location_y.unique())

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and subplots
fig, ax = plt.subplots(1, 1, figsize=(9, 9))

# Create a box plot of all the columns
columns_list = ['euclidean_distance_xz']
sns.boxplot(euclidean_distance_df[columns_list], ax=ax)

# Label the y-axis
ax.set_ylabel('Nearest Horizontal Distance (Euclidean)')

plt.show()

In [ ]:

# Create a figure and subplots
fig, ax = plt.subplots(1, 1, figsize=(9, 9))

# Create a box plot of all the columns
mask_series = (euclidean_distance_df.euclidean_distance_xz < 20)
columns_list = ['euclidean_distance_xz']
sns.boxplot(euclidean_distance_df[mask_series][columns_list], ax=ax)

# Label the y-axis
ax.set_ylabel('Nearest Horizontal Distance (Euclidean)')

plt.show()

In [ ]:

euclidean_distance_df.sort_values('euclidean_distance_xz', ascending=False).head()